
#UK Single Use Product (SUP): Eligibility Expansion Impact Analysis
## AUthor Joby George (jobyg)
## 4/10/2025

## Context: Current Operating SUP Eligibility Strategy
- Customer Risk Segmentation:
  - **Low Risk**: ~90% of customer base
    - Full access to all SUP merchants (including Amazon/eBay)
  
  - **High Risk**: ~10% of customer base
    - Limited SUP merchant access
    - Cannot access Amazon/eBay

## Experiment

## Test Design (Launch: April 16, 2025)
### Population Split
1. **Control Group** (10% of all customers)
   - Maintains current risk-based restrictions
   - High risk users continue to have limited merchant access

2. **Treatment Group A** (80% of all customers)
   - All customers get full merchant access
   - Standard transaction controls

3. **Treatment Group B** (10% of all customers)
   - All customers get full merchant access
   - Enhanced transaction privileges

## Smoke Test (Launch April 9th, 2025)

On April 9th, a proof of concept was done to confirm the test roll-out would work as intended. The Smoke-Test kept 80% of customers as BAU, with 20% given full merchant access. Out of the enabled 20%, 10% of them were also given trusted layer priviledges.



# Script Goals:

This script looks to measure the experiment effects on consumer purchasing behavior looking at the following:

## 1. Core Impact Analysis - High Risk Population
### Primary Metrics
- **Attempt Rate**
  - High Risk Control vs.
  - High Risk Expanded Eligibility vs.
  - High Risk Expanded Eligibility + Reduced Controls
  
- **Fraud Decline Rate**
  - Split by population groups above
  - Control for existing fraud patterns
  
- **GPV (Gross Payment Volume)**
  - Total volume
  - Average transaction size
  - Transaction frequency
  
- **NAC (New Active Customer) Rate**
  - First-time SUP usage
  - Activation patterns by group

### Control Variables
- Existing trusted layer assignments

## 2. Transaction Control Validation
### Technical Verification
- Confirm no overlap between:
  - Reduced transaction control group
  - GB trusted layer rule assignments
  
### Population Segmentation
- Create clean comparison groups:
  1. Base: High Risk Control
  2. Treatment 1: High Risk + Expanded Eligibility (only)
  3. Treatment 2: High Risk + Expanded Eligibility + Reduced Controls
     - Exclude any GB trusted layer assignments

## 3. Risk Tier Analysis for Reduced Controls
### Comparison Framework
Split Treatment 2 (Expanded + Reduced Controls) by:
- Low Risk
  - With existing trusted layer
  - Without existing trusted layer
- High Risk
  - With existing trusted layer
  - Without existing trusted layer

### Metrics by Segment
- Attempt success rate
- Transaction volume
- Fraud rates (takes time to mature)
- Decline rates


#Prepping the data to compare impact of expanded merchant eligibility

In [0]:
#set up

import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# from utils import policy_dict, policy_data_dict, find_non_zero_riskweight_rules, find_zero_riskweight_rules
from pandasql import sqldf
from tqdm import tqdm
import logging

# Standard data manipulation and visualization packages
import pandas as pd
import numpy as np


# Display settings
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)      # Show 100 rows max
pd.set_option('display.width', None)        # Auto-detect display width
pd.set_option('display.float_format', lambda x: '%.3f' % x)  # Format floats to 3 decimal places

# Plotting settings
plt.style.use('seaborn')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 6]  # Set default figure size


from pysnowflake import Session
run_query = lambda query: sqldf(query, globals())




In [0]:
user_name = 'jobyg' #replace it with your ldap name
sess = Session(
   connection_override_args={
       'autocommit': True,
       'authenticator': 'externalbrowser',
       'account': 'square',
       'database': f'PERSONAL_{user_name.upper()}',
       'user': f'{user_name}@squareup.com'
   }
   
)
conn = sess.open()



In [0]:
conn.execute('use warehouse ADHOC__LARGE')
conn.execute('use database AP_CUR_FRDRISK_G')
conn.execute('use schema public')
rt_start_time =  (datetime.now().today()- timedelta(days=2)).strftime("%Y-%m-%d")

print(rt_start_time) 

In [0]:
historical_start_time = '2025-04-09 19:03:38.711' 
#this is the time period we'll loo at for the analysis
rt_start_time =  (datetime.now().today()- timedelta(days=2)).strftime("%Y-%m-%d")

historical_table_name = f'ap_cur_r_featsci.curated_feature_science_red.tbl_raw_r_e_rekarma_rl_exec_rslt_rules__{user_name}_dsl3_sv'
historical_time_col = 'event_info_event_time'

rt_table_name = f'ap_cur_r_featsci.curated_feature_science_red.raw_r_e_rekarma_rl_exec_rslt_rules_rt__{user_name}_dsl3_sv'
rt_time_col = 'event_time'

rule = 'gb_fraud_online_sup_eligibility_model_risk_filter_elgibility_experiment'
##pull from the historical table first
historical_q = f''' create or replace temp table SUP_risk_filter_events AS (
SELECT 
    a.order_token,
    a.consumer_uuid,
    a.par_region, 
    a.par_process_date, 
    a.checkpoint,
    a.{historical_time_col} as event_time,
    a.rule_id, 
    a.rule_name,
    a.rule_external_id,
    a.sub_rules, 
    a.risk_score,
    a.rule_result,
    a.is_rejected,
    a.is_in_treatment,
    a.status,
    a.extra,
    a.rule_category,
    a.status_detail,
    a.actions,
    a.rule_execution_version,
    parse_json((extra))['transient_features']['holdout_group'] as holdout_group,  
    parse_json((extra))['transient_features']['pre_expansion_risk_tier'] as         pre_expansion_Risk_Tier,
    parse_json((extra))['transient_features']['test_key'] as test_key,
    c.first_order_date,
    CASE 
        WHEN c.first_order_date IS NULL THEN 'new'
        WHEN DATEDIFF('day', c.first_order_date, a.par_process_date) < 15 THEN 'new'
        ELSE 'existing'
    END as customer_tenure
FROM {historical_table_name} a
LEFT JOIN AP_RAW_GREEN.GREEN.D_CONSUMER c
    ON a.consumer_uuid = c.uuid
WHERE a.{historical_time_col} <= '{rt_start_time}'
and a.{historical_time_col} >= '{historical_start_time}'
and customer_tenure = 'new'
and a.rule_id = '{rule}');'''
conn.execute(historical_q)
#now append rt values to this

insert_query = f'''create or replace temp table SUP_risk_filter_events_insert AS (
SELECT 
    a.order_token,
    a.consumer_uuid,
    a.par_region, 
    a.par_process_date, 
    a.checkpoint,
    a.{rt_time_col},
    a.rule_id, 
    a.rule_name,
    a.rule_external_id,
    a.sub_rules, 
    a.risk_score,
    a.rule_result,
    a.is_rejected,
    a.is_in_treatment,
    a.status,
    a.extra,
    a.rule_category,
    a.status_detail,
    a.actions,
    a.rule_execution_version,
    parse_json((extra))['transient_features']['holdout_group'] as holdout_group,  
    parse_json((extra))['transient_features']['pre_expansion_risk_tier'] as         pre_expansion_Risk_Tier,
    parse_json((extra))['transient_features']['test_key'] as test_key,
    c.first_order_date,
    CASE 
        WHEN c.first_order_date IS NULL THEN 'new'
        WHEN DATEDIFF('day', c.first_order_date, a.par_process_date) < 15 THEN 'new'
        ELSE 'existing'
    END as customer_tenure
FROM {rt_table_name} a
LEFT JOIN AP_RAW_GREEN.GREEN.D_CONSUMER c
    ON a.consumer_uuid = c.uuid
WHERE a.{rt_time_col} >= '{rt_start_time}'
and customer_tenure = 'new'
and a.rule_id = '{rule}'); '''

conn.execute(insert_query)
# b= conn.download('select * from SUP_risk_filter_events_v2 limit 5')
# c= conn.download('select * from SUP_risk_filter_events_insert limit 5')
# print(b.columns)
# print(c.columns)

conn.execute(f'insert into SUP_risk_filter_events select * from SUP_risk_filter_events_insert')


In [0]:

order_token_driver = f'''create or replace temp table SUP_attempts AS (
    SELECT 
        a.order_token
        ,a.consumer_id
        ,a.checkout_time
        ,a.par_region
        ,a.bp_is_sup
        ,a.consumer_is_First_order
        ,a.days_since_First_order_date
        ,a.in_flight_order_merchant_name
        ,a.in_flight_order_merchant_id
        ,a.trust as featuremart_trust
        ,case when d.order_token is not null then 1 else 0 end as original_gb_trust_layer
        ,case when e.order_token is not null then 1 else 0 end as gb_trust_expansion
        ,a.in_flight_order_amount
        ,a.order_amount_local
        ,a.whitepages_identity_check_score
        ,a.bp_c_batch_consumer_batch_model_v1
        ,a.MODEL_ONLINE_OD_ABUSIVE_GLOBAL_APRIL_2024_SCORE
        ,a.MODEL_ONLINE_OD_PAYBACK_NON_US_APRIL_2024_SCORE
        ,a.MODEL_ONLINE_OD_GLOBAL_PAYBACK_SUP_MODEL_AUGUST_2023_SCORE
        ,c.holdout_group
        ,min_daily_risk_tier
        ,max_daily_risk_tier
        ,case when b.min_daily_risk_tier = 0 then 'L' else 'H' end as min_daily_risk_tier_fixed
        ,case when b.max_daily_risk_tier = 0 then 'L' else 'H' end as max_daily_risk_tier_fixed
        ,case when bp_c_batch_consumer_batch_model_v1 <= 780 then 'L' else 'H' end as order_risk_tier
        ,case when b.min_daily_risk_tier = b.max_daily_risk_tier then 1 else 0 end as daily_risk_tier_was_consistent
        from  ap_cur_r_frdrisk.curated_fraud_risk_red.unified_feature_datamart_base__{user_name}_dsl3_sv a
        left join eligibility_driver_cust_day_deduped b
        on a.consumer_id = b.consumer_uuid
        and a.par_process_Date =b.par_process_date
        left join holdouts_deduped c
        on a.consumer_id = c.consumer_uuid
        left join gb_trusted_layer_v2 d
        on a.order_token = d.order_token
        left join eligibility_trust_layer e
        on a.order_token = e.order_token
        where a.par_Region = 'GB'
        and a.bp_is_sup = 1
        and coalesce(a.days_since_First_order_date,0) <= 14
        and dedup=1
        and checkout_time >= '{historical_start_time}'        
        );
'''
conn.execute(order_token_driver)



In [0]:
def create_eligibiltiy_drivers_for_analysis(user_name, conn=conn):
    """
    Create drivers for data analysis based on user data tables.

    This function generates analysis drivers using either real-time or historical tables
    depending on the specified parameters.

    Parameters
    ----------
    start_time : datetime
        The starting time point for the analysis.
    
    user_name : str
        Username used to specify the relevant user data tables for analysis.
    
    rt : bool
        Real-time flag:
        - If True, uses real-time tables for analysis
        - If False, uses historical tables for analysis
    
    conn : connection object, optional
        Database connection object. Defaults to global 'conn' variable.

    Returns
    ----------
    Temporary SQL tables  are used for further analysis, for a given rule, using rt source tables if specified 
    """
    #lets first start with the historical records for the rule and then append the rt
    historical_start_time = '2025-04-09 19:03:38.711' 
    #this is the time period we'll loo at for the analysis
    rt_start_time =  (datetime.now().today()- timedelta(days=2)).strftime("%Y-%m-%d")
    historical_table_name = f'ap_cur_r_featsci.curated_feature_science_red.tbl_raw_r_e_rekarma_rl_exec_rslt_rules__{user_name}_dsl3_sv'
    historical_time_col = 'event_info_event_time'

    rt_table_name = f'ap_cur_r_featsci.curated_feature_science_red.raw_r_e_rekarma_rl_exec_rslt_rules_rt__{user_name}_dsl3_sv'
    rt_time_col = 'event_time'
    rule = 'gb_fraud_online_sup_eligibility_model_risk_filter_elgibility_experiment'

    ##pull from the historical table first
    print('grabbing all historical SUP eligibility events')
    historical_q = f''' create or replace temp table SUP_risk_filter_events AS (
SELECT 
    a.order_token,
    a.consumer_uuid,
    a.par_region, 
    a.par_process_date, 
    a.checkpoint,
    a.{historical_time_col} as event_time,
    a.rule_id, 
    a.rule_name,
    a.rule_external_id,
    a.sub_rules, 
    a.risk_score,
    a.rule_result,
    a.is_rejected,
    a.is_in_treatment,
    a.status,
    a.extra,
    a.rule_category,
    a.status_detail,
    a.actions,
    a.rule_execution_version,
    parse_json((extra))['transient_features']['holdout_group'] as holdout_group,  
    parse_json((extra))['transient_features']['pre_expansion_risk_tier'] as         pre_expansion_Risk_Tier,
    parse_json((extra))['transient_features']['test_key'] as test_key,
    c.first_order_date,
    CASE 
        WHEN c.first_order_date IS NULL THEN 'new'
        WHEN DATEDIFF('day', c.first_order_date, a.par_process_date) < 15 THEN 'new'
        ELSE 'existing'
    END as customer_tenure
FROM {historical_table_name} a
LEFT JOIN AP_RAW_GREEN.GREEN.D_CONSUMER c
    ON a.consumer_uuid = c.uuid
WHERE a.{historical_time_col} <= '{rt_start_time}'
and a.{historical_time_col} >= '{historical_start_time}'
and customer_tenure = 'new'
and a.rule_id = '{rule}');'''
    conn.execute(historical_q)

#now append rt values to this

    insert_query = f'''create or replace temp table SUP_risk_filter_events_insert AS (
SELECT 
    a.order_token,
    a.consumer_uuid,
    a.par_region, 
    a.par_process_date, 
    a.checkpoint,
    a.{rt_time_col},
    a.rule_id, 
    a.rule_name,
    a.rule_external_id,
    a.sub_rules, 
    a.risk_score,
    a.rule_result,
    a.is_rejected,
    a.is_in_treatment,
    a.status,
    a.extra,
    a.rule_category,
    a.status_detail,
    a.actions,
    a.rule_execution_version,
    parse_json((extra))['transient_features']['holdout_group'] as holdout_group,  
    parse_json((extra))['transient_features']['pre_expansion_risk_tier'] as         pre_expansion_Risk_Tier,
    parse_json((extra))['transient_features']['test_key'] as test_key,
    c.first_order_date,
    CASE 
        WHEN c.first_order_date IS NULL THEN 'new'
        WHEN DATEDIFF('day', c.first_order_date, a.par_process_date) < 15 THEN 'new'
        ELSE 'existing'
    END as customer_tenure
FROM {rt_table_name} a
LEFT JOIN AP_RAW_GREEN.GREEN.D_CONSUMER c
    ON a.consumer_uuid = c.uuid
WHERE a.{rt_time_col} >= '{rt_start_time}'
and customer_tenure = 'new'
and a.rule_id = '{rule}'); '''

    conn.execute(insert_query)
    print('appending RT SUP eligibility events into historical table')
    
    conn.execute(f'insert into SUP_risk_filter_events select * from SUP_risk_filter_events_insert')


    ## create accompanying driver tables:
    eligbility_driver_cust_day_level_q = conn.execute(f'''
   create or replace temp table eligibility_driver_cust_day_deduped as (select 
        par_process_date
        ,consumer_uuid
        ,min(case when pre_expansion_risk_tier = 'L' then 0 
                  when pre_expansion_risk_tier = 'H' then 1 end) as min_daily_risk_tier
        ,max(case when pre_expansion_risk_tier = 'L' then 0 
                  when pre_expansion_risk_tier = 'H' then 1 end) as max_daily_risk_tier
        ,max(holdout_group) as holdout_group
        from SUP_risk_filter_events
        group by 1,2);''')
    
    holdout_group = conn.execute('create or replace temp table holdouts_deduped as (select distinct consumer_uuid, par_process_date, max(holdout_group) as holdout_group from SUP_risk_filter_events group by 1,2);')

    print('grabbing trusted layer events')

    trusted_layer_historical = f'''create or replace temp table gb_trusted_layer_v2 as (
 SELECT order_token, rule_id, EVENT_INFO_EVENT_TIME
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.TBL_RAW_C_E_FC_DECISION_RECORD_RULES__{user_name}_DSL3_SV
    WHERE checkpoint in ('CHECKOUT_CONFIRM')
    AND (actions!='[]')
    AND to_timestamp(EVENT_INFO_EVENT_TIME/1000) <= '{rt_start_time}'
    AND to_timestamp(EVENT_INFO_EVENT_TIME/1000) > '{historical_start_time}'
    and RULE_ID in ('gb_trusted_layer_v2')
    and par_region in ('GB'));'''
    conn.execute(trusted_layer_historical)
    trusted_layer_rt = f'''create or replace temp table gb_trusted_layer_v2_insert as ( 
 SELECT order_token, rule_id, EVENT_INFO_EVENT_TIME
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.RAW_C_E_FC_DECISION_RECORD_RULES_RT__{user_name}_DSL3_SV
    WHERE checkpoint in ('CHECKOUT_CONFIRM')
    AND (actions!='[]')
    AND par_process_date >= '{rt_start_time}'
    and RULE_ID in ('gb_trusted_layer_v2')
    and par_region in ('GB'));'''
    conn.execute(trusted_layer_rt)

    conn.execute('insert into gb_trusted_layer_v2 select * from gb_trusted_layer_v2_insert')


    trusted_layer_eligibility = f'''create or replace temp table eligibility_trust_layer as (
 SELECT order_token, rule_id, EVENT_INFO_EVENT_TIME
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.TBL_RAW_C_E_FC_DECISION_RECORD_RULES__{user_name}_DSL3_SV
    WHERE checkpoint in ('CHECKOUT_CONFIRM')
    AND (actions!='[]')
    AND to_timestamp(EVENT_INFO_EVENT_TIME/1000) <= '{rt_start_time}'
    AND to_timestamp(EVENT_INFO_EVENT_TIME/1000) > '{historical_start_time}'
    and RULE_ID in ('gb_trusted_layer_v_eligibility_expansion')
    and par_region in ('GB'));'''
    conn.execute(trusted_layer_eligibility)

    trusted_layer_eligibility_rt = f'''create or replace temp table eligibility_trust_layer_insert as (
    SELECT order_token, rule_id, EVENT_INFO_EVENT_TIME
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.RAW_C_E_FC_DECISION_RECORD_RULES_RT__{user_name}_DSL3_SV
    WHERE checkpoint in ('CHECKOUT_CONFIRM')
    AND (actions!='[]')
    AND par_process_date >= '{rt_start_time}'
    and RULE_ID in ('gb_trusted_layer_v_eligibility_expansion')
    and par_region in ('GB'));'''
    conn.execute(trusted_layer_eligibility_rt)

    conn.execute('insert into eligibility_trust_layer select * from eligibility_trust_layer_insert')

    decline_flag_historical = f'''create or replace temp table decline_flag_historical as (
 SELECT order_token, rule_id, EVENT_INFO_EVENT_TIME
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.TBL_RAW_C_E_FC_DECISION_RECORD_RULES__{user_name}_DSL3_SV
    WHERE checkpoint in ('CHECKOUT_CONFIRM')
    AND ((is_rejected = 'True' and is_in_treatment ilike 'True'))    
    AND to_timestamp(EVENT_INFO_EVENT_TIME/1000) <= '{rt_start_time}'
    AND to_timestamp(EVENT_INFO_EVENT_TIME/1000) > '{historical_start_time}'
    and par_region in ('GB'));'''
    conn.execute(decline_flag_historical)

    decline_flag_rt = f'''create or replace temp table decline_flag_rt as (
    SELECT order_token, rule_id, EVENT_INFO_EVENT_TIME
    FROM  AP_CUR_R_FEATSCI.curated_feature_science_red.RAW_C_E_FC_DECISION_RECORD_RULES_RT__{user_name}_DSL3_SV
    WHERE checkpoint in ('CHECKOUT_CONFIRM')
    AND ((is_rejected = 'True' and is_in_treatment ilike 'True'))    
    AND par_process_date >= '{rt_start_time}'
    and par_region in ('GB'));'''
    conn.execute(decline_flag_rt)

    conn.execute('insert into decline_flag_historical select * from decline_flag_rt')

    order_token_driver = f'''create or replace table ap_cur_frdrisk_g.public.{user_name}_GB_SUP_eligibility_expansion_base AS (
    SELECT distinct
        a.order_token
        ,a.consumer_id
        ,a.checkout_time
        ,a.par_region
        ,a.bp_is_sup
        ,a.consumer_is_First_order
        ,a.days_since_First_order_date
        ,a.in_flight_order_merchant_name
        ,a.in_flight_order_merchant_id
        ,a.trust as featuremart_trust
        ,case when d.order_token is not null then 1 else 0 end as original_gb_trust_layer
        ,case when e.order_token is not null then 1 else 0 end as gb_trust_expansion
        ,a.in_flight_order_amount
        ,a.order_amount_local
        ,a.whitepages_identity_check_score
        ,a.bp_c_batch_consumer_batch_model_v1
        ,a.MODEL_ONLINE_OD_ABUSIVE_GLOBAL_APRIL_2024_SCORE
        ,a.MODEL_ONLINE_OD_PAYBACK_NON_US_APRIL_2024_SCORE
        ,a.MODEL_ONLINE_OD_GLOBAL_PAYBACK_SUP_MODEL_AUGUST_2023_SCORE
        ,c.holdout_group
        ,min_daily_risk_tier
        ,max_daily_risk_tier
        ,case when b.min_daily_risk_tier = 0 then 'L' else 'H' end as min_daily_risk_tier_fixed
        ,case when b.max_daily_risk_tier = 0 then 'L' else 'H' end as max_daily_risk_tier_fixed
        ,case when bp_c_batch_consumer_batch_model_v1 <= 780 then 'L' else 'H' end as order_risk_tier
        ,case when b.min_daily_risk_tier = b.max_daily_risk_tier then 1 else 0 end as daily_risk_tier_was_consistent
        ,case when f.order_token is not null then 1 else 0 end as fc_decline_Flag
        ,g.status
        ,g.status_reason
        ,h.order_id
        ,h.amplitude_id
        from  ap_cur_r_frdrisk.curated_fraud_risk_red.unified_feature_datamart_base__{user_name}_dsl3_sv a
        left join eligibility_driver_cust_day_deduped b
        on a.consumer_id = b.consumer_uuid
        and a.par_process_Date =b.par_process_date
        left join holdouts_deduped c
        on a.consumer_id = c.consumer_uuid
        and a.par_process_date = c.par_process_date
        left join gb_trusted_layer_v2 d
        on a.order_token = d.order_token
        left join eligibility_trust_layer e
        on a.order_token = e.order_token
        left join decline_flag_historical f
        on a.order_token = f.order_token
        left join ap_raw_green.green.raw_c_e_order g
        on a.order_token = g.token
        left join (select distinct transaction_token, order_id, amplitude_id from ap_cur_bi_g.curated_analytics_green.cur_c_m_attempt_master) h
        on a.order_token = h.TRANSACTION_TOKEN
        and a.par_region = h.par_Region
        where a.par_Region = 'GB'
        and a.bp_is_sup = 1
        and coalesce(a.days_since_First_order_date,0) <= 14
        and dedup=1
        and checkout_time >= '{historical_start_time}'        
        );
'''
    conn.execute(order_token_driver)
    
    



In [0]:
create_eligibiltiy_drivers_for_analysis('jobyg',conn=conn)

In [0]:
validation = conn.download(f'select order_token, count(*) as ct from ap_cur_frdrisk_g.public.{user_name}_GB_SUP_eligibility_expansion_base group by 1 order by 2 desc limit 10')
validation

In [0]:
start_time = '2025-04-09 19:03:38.711'
rule_id = 'gb_fraud_online_sup_eligibility_model_risk_filter_elgibility_experiment'
#create tables:
param_dict = {0:  
        {'rule_id':"gb_fraud_online_sup_eligibility_model_risk_filter_elgibility_experiment",
        'rt':True},
              1:
        {'rule_id':"gb_fraud_online_sup_eligibility_model_risk_filter_elgibility_experiment",'rt':False},  
        2:{'rule_id':None,'rt':True},  
        3:{'rule_id':None,'rt':False},          
              }
for v in param_dict.values():
    rule_id = v['rule_id']
    rt_bool = v['rt']
    create_drivers_for_analysis(start_time=start_time,
                            user_name=user_name,
                            rule=rule_id,
                            rt=rt_bool,
                            )


#now with all my tables i can create various drivers 

In [0]:
if rt:
            time_col = 'event_time'
            source_table = f'ap_cur_r_featsci.curated_feature_science_red.raw_r_e_rekarma_rl_exec_rslt_rules_rt__{user_name}_dsl3_sv'
    output_table = 'SUP_risk_filter_events_rt'
        else:
            source_table = 
            output_table = 'SUP_risk_filter_events'
            time_col = 'event_info_event_time'
        query = f'''
    create or replace temp table {output_table} AS (
    SELECT 
        a.*,
         parse_json((extra))['transient_features']['holdout_group'] as holdout_group,  
    parse_json((extra))['transient_features']['pre_expansion_risk_tier'] as pre_expansion_Risk_Tier,
    parse_json((extra))['transient_features']['test_key'] as test_key,
        c.first_order_date,
        CASE 
            WHEN c.first_order_date IS NULL THEN 'new'
            WHEN DATEDIFF('day', c.first_order_date, a.par_process_date) < 15 THEN 'new'
            ELSE 'existing'
        END as customer_tenure
    FROM {source_table} a
    LEFT JOIN AP_RAW_GREEN.GREEN.D_CONSUMER c
        ON a.consumer_uuid = c.uuid
    WHERE a.{time_col} >= '{start_time}'
    and customer_tenure = 'new'
    and a.rule_id = '{rule}'
)'''

In [0]:
#check if a consumer has multiple risk_tiers per day
risk_tier_dupe_check = conn.download('select par_process_Date, consumer_uuid, count(distinct(pre_expansion_risk_tier)) as risk_tier_count from SUP_risk_filter_events group by 1,2 order by 3 desc limit 5')
risk_tier_dupe_check ## so a customer can be evaluated as having both low and high risk tiers on the same day, this will be a bit tricky to find the high risk population, but i'll use aggregates to paint a clearer picture


In [0]:
eligbility_driver_cust_day_level_q = conn.execute('''
   create or replace temp table eligibility_driver_cust_day_deduped as (select 
        par_process_date
        ,consumer_uuid
        ,min(case when pre_expansion_risk_tier = 'L' then 0 
                  when pre_expansion_risk_tier = 'H' then 1 end) as min_daily_risk_tier
        ,max(case when pre_expansion_risk_tier = 'L' then 0 
                  when pre_expansion_risk_tier = 'H' then 1 end) as max_daily_risk_tier
        ,max(holdout_group) as holdout_group
        from SUP_risk_filter_events
        group by 1,2)''')
    


In [0]:
holdout_group = conn.execute('create or replace temp table holdouts_deduped as (select distinct consumer_uuid, holdout_group from SUP_Risk_Filter_Events);')

In [0]:
conn.download('select * from holdouts_deduped')

In [0]:
#check if a consumer has multiple risk_tiers per day
holdout_group_dupe_check = conn.download('select consumer_uuid, count(distinct(holdout_group)) as holdout_group_count from holdouts_deduped group by 1 order by 2 desc limit 5')
holdout_group_dupe_check ## so a customer can be evaluated as having both low and high risk tiers on the same day, this will be a bit tricky to find the high risk population, but i'll use aggregates to paint a clearer picture


In [0]:
#CHECKOUT TIME filter needs to be applied as to make sure holdout group is properly applied
order_token_driver = '''create or replace temp table SUP_attempts AS (
    SELECT 
        a.order_token
        ,a.consumer_id
        ,a.checkout_time
        ,a.par_region
        ,a.bp_is_sup
        ,a.consumer_is_First_order
        ,a.days_since_First_order_date
        ,a.in_flight_order_merchant_name
        ,a.in_flight_order_merchant_id
        ,a.trust
        ,a.in_flight_order_amount
        ,a.order_amount_local
        ,a.whitepages_identity_check_score
        ,a.bp_c_batch_consumer_batch_model_v1
        ,a.MODEL_ONLINE_OD_ABUSIVE_GLOBAL_APRIL_2024_SCORE
        ,a.MODEL_ONLINE_OD_PAYBACK_NON_US_APRIL_2024_SCORE
        ,a.MODEL_ONLINE_OD_GLOBAL_PAYBACK_SUP_MODEL_AUGUST_2023_SCORE
        ,c.holdout_group
        ,min_daily_risk_tier
        ,max_daily_risk_tier
        ,case when b.min_daily_risk_tier = 0 then 'L' else 'H' end as min_daily_risk_tier_fixed
        ,case when b.max_daily_risk_tier = 0 then 'L' else 'H' end as max_daily_risk_tier_fixed
        ,case when bp_c_batch_consumer_batch_model_v1 <= 780 then 'L' else 'H' end as order_risk_tier
        ,case when b.min_daily_risk_tier = b.max_daily_risk_tier then 1 else 0 end as daily_risk_tier_was_consistent
        from  ap_cur_r_frdrisk.curated_fraud_risk_red.unified_feature_datamart_base__jobyg_dsl3_sv a
        left join eligibility_driver_cust_day_deduped b
        on a.consumer_id = b.consumer_uuid
        and a.par_process_Date =b.par_process_date
        left join holdouts_deduped c
        on a.consumer_id = c.consumer_uuid
        where a.par_Region = 'GB'
        and a.par_process_date >= '2025-04-09'
        and a.bp_is_sup = 1
        and coalesce(a.days_since_First_order_date,0) <= 14
        and dedup=1
        and checkout_time >= '2025-04-09 21:03:38.711'        );
'''
conn.execute(order_token_driver)

In [0]:
#confirm each order_token has a holdoutgroup
view = conn.download('select * from SUP_attempts')
print(view.shape)


In [0]:
view.loc[view.holdout_group.isnull()] #edge cases where the eligibility tracker happened in between rule modifications, i can remove them

In [0]:
view = view.loc[~view.holdout_group.isnull()]

In [0]:
view.loc[view.min_daily_risk_tier.isnull()] #ok the min and max daily risk tier are null due to the join logic as they occur right at midnight >.>, i'll remove them for now 

In [0]:
view = view.loc[~view.min_daily_risk_tier.isnull()]

In [0]:
view.shape #removed 14 transactions in total, 1.8% impact but makes my analysis clean


# Let's Compare High Risk BAU Control to High risk expanded elgibility

In [0]:
high_risk_pop = conn.download('''select * from SUP_attempts where (min_daily_risk_tier_fixed = 'H' or max_daily_risk_tier_fixed = 'H' or order_risk_tier = 'H') and holdout_Group is not null and min_daily_risk_tier is not null ''')

In [0]:
high_risk_pop.daily_risk_tier_was_consistent.value_counts(normalize=True) #for the most part, all the high risk attempts were made by consistently high risk customers throughout the day, let's look at where the flag = 0

In [0]:
high_risk_pop.loc[high_risk_pop.daily_risk_tier_was_consistent==0] #since all of the flags where the daily risk tier was consistent had an order_risk_Tier of H, we'll label all of these as high risk tiers, using the max tier

In [0]:
analysis = conn.download('''
        select 
            max_daily_risk_tier_fixed, 
            holdout_group, 
            in_flight_order_merchant_name,
            count(distinct(order_token)) as token_ct,
            sum(in_flight_order_amount)  as attempt_amt
            from
            SUP_attempts where max_daily_risk_tier_fixed = 'H'
            and holdout_Group is not null
            group by 1,2,3
             ''')
analysis.sort_values(by =['holdout_group', 'token_ct'], ascending=[True,False]) #uh oh how are the high risk customers placing orders at amazon

In [0]:
amazon = conn.download('''select * from SUP_attempts where in_flight_order_merchant_name ilike 'AMAZON%' ''')

In [0]:
amazon.holdout_group.value_counts(normalize=True)

In [0]:
examination.loc[examination.checkpoint == 'SUP_MERCHANT_RISK_FILTER']

In [0]:
else:
        if rt:
            source_table = f'AP_CUR_R_FEATSCI.curated_feature_science_red.TBL_RAW_C_E_FC_DECISION_RECORD_RULES__{user_name}_DSL3_SV'
            output_table = 'trusted_layer_validation_rt'
        
        else:
            source_table = f'AP_CUR_R_FEATSCI.curated_feature_science_red.RAW_C_E_FC_DECISION_RECORD_RULES_RT__{user_name}_DSL3_SV'
            output_table = 'trusted_layer_Validation'
        query = f'''CREATE or replace temp table {output_table} AS (SELECT
        a.*, to_number(substring(MD5(concat(a.CONSUMER_ID,'oon_expansion')),1,6),'XXXXXX')/to_number('ffffff','XXXXXX')::decimal(10,2) as test_key,
        case when merchant_id in ('400538723', '400570637', '400575272', '400568202', '400504709', '400535118', '400551784', '400527714', '400515891', '400546925', '400527604', '400562359', '400474538', '400571508', '400515894', '400569331', '400572942', '400547364', '400474394', '400527759', '400575150', '400562345', '400570636', '400527737', '400527047', '400474401', '400475455', '400527639', '400527593', '400574444', '400461110', '400488351', '400507556', '400527756', '400569337', '400527058', '400570624', '400510052', '400510042', '400515896', '400575275', '400527704', '400514646', '400527833', '400522690', '400527601', '400562315', '400579004', '400527066', '400507557', '400509583', '400507558', '400507559',   '400517492') then 1 else 0 end as gb_sup_flag,
        CASE 
            WHEN c.first_order_date IS NULL THEN 'new'
            WHEN DATEDIFF('day', c.first_order_date, a.par_process_date) <= 14 THEN 'new'
            ELSE 'existing'
        END as customer_tenure
        FROM  {source_table} a
        LEFT JOIN AP_RAW_GREEN.GREEN.D_CONSUMER c
        ON a.CONSUMER_ID = c.uuid
        WHERE a.checkpoint in ('CHECKOUT_CONFIRM')
        -- AND ((is_rejected = 'True' and is_in_treatment ilike 'True'))
        AND a.par_process_date >= '{start_time}'::date
        and a.par_Region = 'GB'); '''


# Look into the trusted layer population

There's two things i need to do, make this more flexibile for a real time vs non real time analysis.